A couple of notes after looking at yesterday's results:

In [1]:
import numpy as np

In [26]:
# np.ndarray just takes a bunch of memory and 
# will from now on use it as array of the given size.
# You need to fill it with useful values first!
a = np.ndarray(12)
a # here we are just lucky that the memory it took contained zeros!

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [30]:
b = np.zeros(12)
b

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [31]:
c = np.ones(12)
c

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [32]:
d = np.random.rand(12)
d

array([ 0.51836592,  0.81728169,  0.82662425,  0.78116571,  0.38554844,
        0.80489915,  0.64218978,  0.44059752,  0.71647987,  0.34321394,
        0.04493703,  0.69608908])

In [50]:
e = np.arange(12)
e

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [51]:
np.array([1,2,3])

numpy.ndarray

**All of the above set up an array that you can immediately use, <br>
all but the first also write something into the array**

## Code duplication is evil

In [54]:
def conv(f,g):
    if type(f) is not np.ndarray or type(g) is not np.ndarray:
        print("ungültiger Datentyp")
        return
    if len(f.shape) != 1 or len(g.shape) != 1:
        print("ungültige Dimension")
        return
        
    s1 = f.shape[0]
    s2 = g.shape[0]
    
    if s1 <= s2:
        x = np.zeros(s2)
        for n in range(0, s2):
            for m in range(0, s1):
                x[n] += f[m]*g[n-m]
        return x
        
    elif s1 > s2: #convolution is commutative 
#         x = np.zeros(s1)
#         for n in range(0, s1):
#             for m in range(0, s2):
#                 x[n] += f[m-n]*g[m]
#         return x
        return conv(g,f)

In [55]:
f = np.zeros(5)
f[0] = 1
g = np.zeros(5)
g[0] = 0.9
g[1] = 0.1

In [56]:
conv(f,g)

array([ 0.9,  0.1,  0. ,  0. ,  0. ])

## Prevent code from being executed more often than needed

In [42]:
def assimilatearrays(f,g):          # füllt kürzeren Vektor mit Nullen auf
    if len(g)<len(f):
        G=np.zeros(len(f))
#         for i in range(len(g)):
#             G[i]=g[i]
        G[0:len(g)] = g # this line does the same as the one before!
        F=f
        return F, G
    else:
        F=np.zeros(len(g))
        for i in range(len(f)):
            F[i]=f[i]
        G=g
        return F,G

def convolution(f,g):           # Faltung zweier Vektoren gleicher Dimension
    N=len(assimilatearrays(f,g)[0])   
    s=np.zeros(N)
    conv=np.zeros(N)
    for n in range(N):
        for i in range(N):
            s[i]=assimilatearrays(f,g)[0][i]*assimilatearrays(f,g)[1][n-i]
        conv[n]=sum(s)
    return conv

In [46]:
f = np.zeros(10**3)
f[0] = 1
g = np.zeros(10**2)
g[0:2] = 0.9, 0.1

In [47]:
%timeit convolution(f,g)

1 loops, best of 3: 36.8 s per loop


In [48]:
def convolutionAssimilateOnce(f,g):           # Faltung zweier Vektoren gleicher Dimension
    F, G = assimilatearrays(f,g)
    N=len(F)   
    s=np.zeros(N)
    conv=np.zeros(N)
    for n in range(N):
        for i in range(N):
            s[i]=F[i]*G[n-i]
        conv[n]=sum(s)
    return conv

In [49]:
%timeit convolutionAssimilateOnce(f,g)

1 loops, best of 3: 437 ms per loop


What happened here? ```assimilatearrays``` was called 2 \* 1000 \* 1000 + 1 times instead of just once!